## [Machine Learning, NLP: Text Classification using scikit-learn, python and NLTK](https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a)

Ejemplo de **Medium.com**, adaptado para la mejor comprensión de la clasificación de textos

In [1]:
from sklearn.datasets import fetch_20newsgroups

El conjunto de datos de 20 grupos de noticias es una colección de aproximadamente 20,000 documentos de grupos de noticias, divididos (casi) de manera uniforme en 20 grupos de noticias diferentes

_______________________

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
from sklearn.model_selection import GridSearchCV

In [5]:
from sklearn.pipeline import Pipeline

In [4]:
from sklearn.naive_bayes import MultinomialNB

In [7]:
from sklearn.linear_model import SGDClassifier

In [11]:
from nltk.stem.snowball import SnowballStemmer

In [6]:
import numpy as np

__________________________________________________

Cargando los datos de entrenamiento

In [12]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

Listado de **Target Names**, es decir las **categorias** en las que estan dividido el twenty_train

In [13]:
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [14]:
print("\n".join(twenty_train.data[0].split("\n")[:3])) 

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


------------------------------------

Los archivos de texto son en realidad series de palabras, para ejecutar algoritmos de aprendizaje automático, se necesita convertir los archivos de texto en vectores de características numéricas.<br><br>
Para esto se utilizara el modelo de **Bag of Words:**<br>
El texto (como una oración o un documento) se representa como la bolsa de sus palabras, sin tener en cuenta la gramática e incluso ni el orden de las palabras, pero manteniendo la cantidad<br>
El modelo de **Bag of Words** se usa comúnmente en los métodos de clasificación de documentos donde la (frecuencia de) aparición de cada palabra se usa como una característica para entrenar a un clasificador (Wikipedia)<br><br>

Cada palabra única en el diccionario corresponderá a una característica (descriptive feature)

-----------------------------

### [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

Convierte una colección de documentos en una matriz de tokens sumarizados

In [15]:
count_vect = CountVectorizer()

In [49]:
X_train_counts = count_vect.fit_transform(twenty_train.data) 

Aprende el vocabulario y devuelve la matriz de términos<br>
Esto es equivalente al fit, pero implementado de manera más eficiente

In [50]:
X_train_counts.shape

(11314, 130107)

indice con terminos

In [56]:
count_vect.vocabulary_.get('document')

48411

##### [Text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

______________________________

### [sklearn.feature_extraction.text.TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html?highlight=tfidftransformer#sklearn.feature_extraction.text.TfidfTransformer)

Solo contar el número de palabras en cada documento tiene un problema:<br>
Dará más peso a documentos más largos que documentos más cortos.<br>
Para evitar esto, se utiliza TF (**Term Frequencies**)<br>
Luego se reduce el peso de las palabras más comunes como ser el, y, un etc. <br>
**TF-IDF** Term Frequency times inverse document frequency

Recibe una matris y la normaliza en la forma tf-idf

In [18]:
tfidf_transformer = TfidfTransformer()

In [19]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

Dimensión de la matriz de términos del documento

In [20]:
X_train_tfidf.shape

(11314, 130107)

_________________________________

### [sklearn.naive_bayes.MultinomialNB](https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a)

In [21]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

____________________

Alternativamente todos los pasos anteriores se pueden reducir con la utilizacion de un **Pipeline**.

### [sklearn.pipeline: Pipeline](https://scikit-learn.org/stable/modules/classes.html?highlight=pipeline#module-sklearn.pipeline)

In [22]:
text_clf = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

In [23]:
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [24]:
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [25]:
predicted = text_clf.predict(twenty_test.data)

In [26]:
np.mean(predicted == twenty_test.target) #accuracy de Naive Bayes

0.7738980350504514

___________________________________

### [Support Vector Machines](htthttps://scikit-learn.org/stable/modules/svm.html)

**Support vector machines** (SVM) son un conjunto de métodos de aprendizaje supervisados classification, regression y outliers detection.

Las ventajas de los SVM son:<br>
  * Efectivo en espacios de altas dimensiones.
  * Sigue siendo efectivo en casos donde el número de dimensiones es mayor que el número de muestras.
  * Utiliza un subconjunto de puntos de entrenamiento en la función de decisión (llamados vectores de soporte), por lo que también es eficiente en la memoria.
  * Versátil: se pueden especificar diferentes funciones de Kernel para la función de decisión. Se proporcionan núcleos comunes, pero también es posible especificar núcleos personalizados.

Las desventajas de los SVM son:<br>
  * Si el número de características es mucho mayor que el número de muestras, evite el ajuste excesivo al elegir las funciones de Kernel y el término de regularización es crucial.
  * Los SVM no proporcionan directamente estimaciones de probabilidad, estas se calculan utilizando una costosa validación cruzada.    

### [sklearn.linear_model.SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html?highlight=sgdclassifier#sklearn.linear_model.SGDClassifier)

**Stochastic gradient descent** (SGD) es un método iterativo para optimizar una función objetivo con propiedades de suavidad adecuadas (por ejemplo una funcion diferencial).<br>
Puede considerarse como una aproximación estocástica de la optimización del **descenso del gradiente**, ya que reemplaza el gradiente real (calculado a partir del conjunto de datos completo) por una estimación del mismo (calculado a partir de un subconjunto de datos seleccionado al azar).<br>
Se utiliza en aplicaciones de **big data**, esto reduce la carga computacional, logrando iteraciones más rápidas en la tasa de convergencia. (Wikipedia)

In [27]:
text_clf_svm = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', 
                                                   penalty='l2', 
                                                   alpha=1e-3, 
                                                   max_iter=7, 
                                                   random_state=42))])

In [28]:
text_clf_svm = text_clf_svm.fit(twenty_train.data, twenty_train.target)

C:\Utils\Anaconda3\envs\austral\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [29]:
predicted_svm = text_clf_svm.predict(twenty_test.data)

In [30]:
np.mean(predicted_svm == twenty_test.target) #accuracy de Naive Bayes

0.8248805098247477

____________________________________

### [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Búsqueda exhaustiva sobre valores de parámetros especificados para un estimador

Se crea una lista de parámetros para los que nos gustaría realizar ajustes de rendimiento.<br>
Todos los parámetros comienzan con el nombre del clasificador (vect, tfidf, clf)<br>
Aquí se configura para unigram y bigrams y elija el que sea óptimo.

___________________

### clf : MultinomialNB

In [62]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 
              'tfidf__use_idf': (True, False), 
              'clf__alpha': (1e-2, 1e-3)}

In [32]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [33]:
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

Mejor Score y Parametros Optimizados

In [34]:
gs_clf.best_score_

0.9157684864695698

In [58]:
gs_clf.best_params_

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

____________________________

### clf-svm :SGDClassifier

In [36]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-3)}

In [37]:
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

In [38]:
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)

C:\Utils\Anaconda3\envs\austral\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


Mejor Score y Parametros Optimizados

In [39]:
gs_clf_svm.best_score_

0.9047198366213406

In [60]:
gs_clf_svm.best_params_

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

_____________________

### Importancia del pre procesamiento

In [41]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

**Stop words**

In [63]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

**Stemming**

In [64]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

In [44]:
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

In [45]:
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), 
                             ('tfidf', TfidfTransformer()), 
                             ('mnb', MultinomialNB(fit_prior=False))])

In [46]:
text_mnb_stemmed = text_mnb_stemmed.fit(twenty_train.data, twenty_train.target)

In [47]:
predicted_mnb_stemmed = text_mnb_stemmed.predict(twenty_test.data)

In [48]:
np.mean(predicted_mnb_stemmed == twenty_test.target)

0.8167817312798725

Sin aplicar Stop Words y Stemming: **0.7738980350504514**<br>
Aplicando  Stop Words y Stemming:  **0.8167817312798725** 

_______________________